In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/saudi-stock-exchange-dividends/Dividends.csv


## Loading Libraries

In [2]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error as MSE

import warnings
warnings.simplefilter("ignore")

## Loading File

In [3]:
ssd=pd.read_csv('/kaggle/input/saudi-stock-exchange-dividends/Dividends.csv')

In [4]:
ssd.shape

(2652, 10)

In [5]:
ssd.head(10)

,symbol,lonaName,shortName,Acronym,Sector,Announced Date,Due Date,Distribution Date,Amount,Distribution Way
0,4061,Anaam International Holding Group,ANAAM HOLDING,ANAAM HOLDING,Food & Staples Retailing,1987-05-12,1987-06-24,1987-07-25,2.5,Account Transfer
1,4061,Anaam International Holding Group,ANAAM HOLDING,ANAAM HOLDING,Food & Staples Retailing,1988-03-20,1988-06-25,1988-07-25,2.5,Account Transfer
2,4100,Makkah Construction and Development Co.,MCDC,MCDC,Real Estate Mgmt & Dev't,1995-02-28,1995-03-29,1995-04-29,5.0,By Mail
3,3020,Yamama Cement Co.,YSCC,YSCC,Materials,1996-02-07,1996-03-09,1996-03-11,37.0,By Mail
4,6010,National Agricultural Development Co.,NADEC,NADEC,Food & Beverages,1996-02-27,1996-02-26,1996-03-18,7.0,By Mail
5,3050,Southern Province Cement Co.,SPCC,SPCC,Materials,1996-03-05,1996-03-21,1996-04-13,30.0,By Mail
6,2020,SABIC Agri-Nutrients Co.,SABIC AGRI-NUTRIENTS,SABIC AGRI-NUTRIENTS,Materials,1996-03-06,1996-03-21,1996-04-04,50.0,By Mail
7,3080,Eastern Province Cement Co.,EPCCO,EPCCO,Materials,1996-03-06,1996-04-02,1996-04-08,35.0,By Mail
8,4100,Makkah Construction and Development Co.,MCDC,MCDC,Real Estate Mgmt & Dev't,1996-03-06,1996-03-13,1996-04-15,7.0,By Mail
9,3040,Qassim Cement Co.,QACCO,QACCO,Materials,1996-03-09,1996-04-04,1996-04-13,30.0,By Mail


In [6]:
ssd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2652 entries, 0 to 2651
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symbol             2652 non-null   int64  
 1   lonaName           2652 non-null   object 
 2   shortName          2652 non-null   object 
 3   Acronym            2652 non-null   object 
 4   Sector             2652 non-null   object 
 5   Announced Date     2652 non-null   object 
 6   Due Date           2637 non-null   object 
 7   Distribution Date  2616 non-null   object 
 8   Amount             2652 non-null   float64
 9   Distribution Way   2652 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 207.3+ KB


## Changing date features into datetime

In [7]:
date_col=['Announced Date','Due Date','Distribution Date']
for date_col_name in date_col:
    ssd[date_col_name]=ssd[date_col_name].astype('datetime64')

In [8]:
ssd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2652 entries, 0 to 2651
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   symbol             2652 non-null   int64         
 1   lonaName           2652 non-null   object        
 2   shortName          2652 non-null   object        
 3   Acronym            2652 non-null   object        
 4   Sector             2652 non-null   object        
 5   Announced Date     2652 non-null   datetime64[ns]
 6   Due Date           2637 non-null   datetime64[ns]
 7   Distribution Date  2616 non-null   datetime64[ns]
 8   Amount             2652 non-null   float64       
 9   Distribution Way   2652 non-null   object        
dtypes: datetime64[ns](3), float64(1), int64(1), object(5)
memory usage: 207.3+ KB


## Handling Missing Data and EDA

In [9]:
ssd[ssd.isnull().values][['Announced Date','Due Date','Distribution Date']]

,Announced Date,Due Date,Distribution Date
42,1997-03-23,NaT,NaT
42,1997-03-23,NaT,NaT
47,1997-04-13,1997-05-07,NaT
51,1997-07-15,NaT,NaT
51,1997-07-15,NaT,NaT
54,1998-01-25,1998-02-15,NaT
100,1999-05-23,1999-05-29,NaT
118,2000-03-27,2000-04-19,NaT
127,2000-05-01,2000-05-25,NaT
150,2001-03-21,2001-04-12,NaT


In [10]:
ssd.dropna(inplace=True)

In [11]:
drop_index=ssd[ssd['Due Date'] < ssd['Announced Date']][['Announced Date','Due Date','Distribution Date']].index.tolist()


In [12]:
ssd.drop(index=drop_index,axis=0,inplace=True)

In [13]:
ssd.head(10)

,symbol,lonaName,shortName,Acronym,Sector,Announced Date,Due Date,Distribution Date,Amount,Distribution Way
0,4061,Anaam International Holding Group,ANAAM HOLDING,ANAAM HOLDING,Food & Staples Retailing,1987-05-12,1987-06-24,1987-07-25,2.5,Account Transfer
1,4061,Anaam International Holding Group,ANAAM HOLDING,ANAAM HOLDING,Food & Staples Retailing,1988-03-20,1988-06-25,1988-07-25,2.5,Account Transfer
2,4100,Makkah Construction and Development Co.,MCDC,MCDC,Real Estate Mgmt & Dev't,1995-02-28,1995-03-29,1995-04-29,5.0,By Mail
3,3020,Yamama Cement Co.,YSCC,YSCC,Materials,1996-02-07,1996-03-09,1996-03-11,37.0,By Mail
5,3050,Southern Province Cement Co.,SPCC,SPCC,Materials,1996-03-05,1996-03-21,1996-04-13,30.0,By Mail
6,2020,SABIC Agri-Nutrients Co.,SABIC AGRI-NUTRIENTS,SABIC AGRI-NUTRIENTS,Materials,1996-03-06,1996-03-21,1996-04-04,50.0,By Mail
7,3080,Eastern Province Cement Co.,EPCCO,EPCCO,Materials,1996-03-06,1996-04-02,1996-04-08,35.0,By Mail
8,4100,Makkah Construction and Development Co.,MCDC,MCDC,Real Estate Mgmt & Dev't,1996-03-06,1996-03-13,1996-04-15,7.0,By Mail
9,3040,Qassim Cement Co.,QACCO,QACCO,Materials,1996-03-09,1996-04-04,1996-04-13,30.0,By Mail
10,2090,National Gypsum Co.,NGC,NGC,Materials,1996-03-13,1996-03-13,1996-05-11,20.0,By Mail


In [14]:
ssd.shape

(2598, 10)

In [15]:
ssd['totaldays']=ssd['Due Date']- ssd['Announced Date']

In [16]:
ssd['distributiondays']= ssd['Distribution Date'] -ssd['Due Date'] 

In [17]:
ssd['duedays']=ssd['totaldays']- ssd['distributiondays'] 

In [18]:
ssd.drop(['Announced Date','Due Date','Distribution Date'],axis=1,inplace=True)

In [19]:
ssd.head()

,symbol,lonaName,shortName,Acronym,Sector,Amount,Distribution Way,totaldays,distributiondays,duedays
0,4061,Anaam International Holding Group,ANAAM HOLDING,ANAAM HOLDING,Food & Staples Retailing,2.5,Account Transfer,43 days,31 days,12 days
1,4061,Anaam International Holding Group,ANAAM HOLDING,ANAAM HOLDING,Food & Staples Retailing,2.5,Account Transfer,97 days,30 days,67 days
2,4100,Makkah Construction and Development Co.,MCDC,MCDC,Real Estate Mgmt & Dev't,5.0,By Mail,29 days,31 days,-2 days
3,3020,Yamama Cement Co.,YSCC,YSCC,Materials,37.0,By Mail,31 days,2 days,29 days
5,3050,Southern Province Cement Co.,SPCC,SPCC,Materials,30.0,By Mail,16 days,23 days,-7 days


In [20]:
day_col=['totaldays','distributiondays','duedays']

In [21]:
for day_col_name in day_col:
    ssd[day_col_name]=ssd[day_col_name].astype('str')

In [22]:
for day_col_name in day_col:
    ssd[day_col_name]=ssd[day_col_name].str.replace('days','')

In [23]:
for day_col_name in day_col:
    ssd[day_col_name]=ssd[day_col_name].astype('int')

In [24]:
ssd['duedays']=ssd['duedays'].apply(lambda value :1 if(value>=0) else 0)

In [25]:
ssd['duedays'].value_counts()

1    1614
0     984
Name: duedays, dtype: int64

In [26]:
ssd['distributiondays'].apply(lambda value :1 if(value>=0) else 0).sum()

2597

In [27]:
ssd.drop(index=ssd[ssd['distributiondays']<0].index,inplace=True)

In [28]:
ssd.shape

(2597, 10)

In [29]:
ssd['Sector'].nunique()

21

In [30]:
ssd['shortName'].nunique()

186

In [31]:
ssd['Distribution Way'].nunique()

3

In [32]:
ssd['symbol'].nunique()

186

## Spliting Data

In [33]:
X_t = ssd.copy()
y_t = X_t.pop('Amount')

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_t, y_t, test_size=0.30, random_state=42)

## Using Target Enconding on Acronym and Sector

In [35]:
from category_encoders import MEstimateEncoder

encoder = MEstimateEncoder(cols=["Acronym","Sector"], m=5)

X_train=encoder.fit_transform(X_train, y_train)

X_test = encoder.transform(X_test)


In [36]:
X_train.drop(['symbol','lonaName','shortName'],axis=1,inplace=True)

In [37]:
X_test.drop(['symbol','lonaName','shortName'],axis=1,inplace=True)

## Handling Distribution way feature into numeric values using label encoder

In [38]:
le=LabelEncoder()
X_train['Distribution Way']=le.fit_transform(X_train['Distribution Way'])
X_test['Distribution Way']=le.transform(X_test['Distribution Way'])

## Hyper Parameters

In [39]:
# params_hyper={
#     'objective':['reg:linear','reg:squarederror'],
#     'n_estimators':[10,40,50,70,80,100,110],
#     'max_depth':[2,3,4,5,6],
#     'learning_rate': [0.01, 0.05, 0.07,0.09,0.1],
#     'colsample_bytree': [0.3,0.5,0.7]
# }

## GridSearch and model building
 commented 

In [40]:
# xg_model=XGBRegressor(seed = 123)
# clf = GridSearchCV(xg_model, params_hyper,
#                    scoring='neg_mean_squared_error', 
#                    verbose=1)
# clf.fit(X_train, y_train)
# print("Best parameters:", clf.best_params_)
# print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

## Model Building

In [41]:
xg_model=XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.09, max_depth= 4, n_estimators= 70, objective='reg:linear',seed = 123)

In [42]:
xg_model.fit(X_train,y_train)

[05:29:44] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.09, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=70, n_jobs=0,
             num_parallel_tree=1, objective='reg:linear', predictor='auto',
             random_state=123, reg_alpha=0, ...)

In [43]:
pred=xg_model.predict(X_test)

## MSE

In [44]:
print(MSE(y_test,pred))

5.855962507925037


## RSMSE

In [45]:
print(np.sqrt(MSE(y_test,pred)))

2.419909607387234


## please upvote me if you like my work and suggestions are welcomed 